In [1]:
!nvidia-smi

Sun Jul  7 11:21:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
dataset_path = "/content/nemotron_test_df.csv"
model_path = "Qwen/Qwen2-7B-Instruct"
#model_path = "01-ai/Yi-1.5-6B-Chat"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [4]:
df = pd.read_csv(dataset_path)
print(df.shape)
df = df.head(1)
df

(83, 10)


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,response
0,19086537,vicuna-33b,llama-2-13b-chat,Can you create a caption using this? The Autum...,"""🍂🎡 The Autumn Bazaar is here! Help Marie by g...",Sure! Here's a caption using the information p...,0,0,1,\n { 'relevance': { 'response': ...


In [5]:
from vllm import LLM, SamplingParams
from vllm.engine.arg_utils import AsyncEngineArgs
from vllm.engine.async_llm_engine import AsyncLLMEngine

In [6]:
sampling_params = SamplingParams(
    n=1,
    best_of=1,
    presence_penalty=0.0,
    frequency_penalty=0.0,
    repetition_penalty=1.0,
    temperature=0.0,
    top_p=1.0,
    top_k=-1,
    min_p=0.0,
    use_beam_search=False,
    length_penalty=1.0,
    early_stopping=False,
    stop=[],
    stop_token_ids=[],
    include_stop_str_in_output=False,
    ignore_eos=False,
    max_tokens=2042,
    logprobs=None,
    prompt_logprobs=None,
    skip_special_tokens=True,
    spaces_between_special_tokens=True
)

In [7]:
model = LLM(
    model=model_path, gpu_memory_utilization=0.9, max_model_len=2048, enable_prefix_caching=False, dtype=torch.float16, trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

WARNING 07-07 11:21:39 config.py:1354] Casting torch.bfloat16 to torch.float16.
INFO 07-07 11:21:39 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='Qwen/Qwen2-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=Qwen/Qwen2-7B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

INFO 07-07 11:25:16 model_runner.py:255] Loading model weights took 14.2487 GB


OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 

In [57]:
prompts = []

for _, entry in tqdm(df.iterrows(), total=len(df)):
    query = entry['prompt']
    response_a = entry['response_a']
    response_b = entry['response_b']
    end = "###END"
    prompt = f"""Act like a discriminator assistant which have the capabilities to distinguish between two responses for the given query.
    You will be given one query and two responses. You have to compare the two responses based on the given correct label and provide feedback like
    which response is more relevant, accurate, clear, comprehensive, concise, logically flowing, evidence-based, consistent, objective, and responsive to the nuances of the query and why.
    If the response A is best then you have to write A, if response B is best then you have to write B and if both are best then you have to write AB.
    The feedback should be detailed and provide clear reasoning for each aspect and in reason compare why one response is better and other in bad.
    Output must be in the following JSON format:
    Output: {{
        "relevance": {{
            "response": "<A/B/AB> Which response is more relevant to the query?",
            "reason": "Provide a detailed explanation of why one response directly addresses the query better and provides more pertinent information."
        }},
        "accuracy": {{
            "response": "<A/B/AB> Which response is more accurate?",
            "reason": "Explain why one response contains more reliable information based on facts and credible sources."
        }},
        "clarity": {{
            "response": "<A/B/AB> Which response is clearer?",
            "reason": "Describe why one response is more straightforward, easy to understand, and free of ambiguity."
        }},
        "completeness": {{
            "response": "<A/B/AB> Which response is more comprehensive?",
            "reason": "Detail why one response covers all aspects of the query more thoroughly."
        }},
        "conciseness": {{
            "response": "<A/B/AB> Which response is more concise?",
            "reason": "Explain why one response is more succinct and to the point without unnecessary information."
        }},
        "logical_flow": {{
            "response": "<A/B/AB> Which response has a better logical flow?",
            "reason": "Discuss why one response presents information in a more organized and coherent manner."
        }},
        "evidence_examples": {{
            "response": "<A/B/AB> Which response provides stronger evidence and examples?",
            "reason": "Specify why one response includes stronger or more relevant evidence and examples to support its points."
        }},
        "consistency": {{
            "response": "<A/B/AB> Which response is more consistent?",
            "reason": "Clarify why one response maintains internal consistency without contradictions."
        }},
        "objectivity": {{
            "response": "<A/B/AB> Which response is more objective?",
            "reason": "Explain why one response is more unbiased and neutral in its presentation."
        }},
        "responsiveness": {{
            "response": "<A/B/AB> Which response is more responsive to the nuances and sub-questions within the query?",
            "reason": "Describe why one response better addresses specific details and aspects of the query, including any sub-questions."
        }}
    }}
    {end}
    Task:
    Query: {query}
    Resonse A: {response_a}
    Response B: {response_b}
    Output: """
    prompts.append(prompt)

  0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
len(prompts)

1

In [59]:
outputs = model.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:56<00:00, 56.22s/it, est. speed input: 18.71 toks/s, output: 17.73 toks/s]


In [60]:
  text = outputs[0].outputs[0].text
  print(text)


     {
         "relevance": {
             "response": "B",
             "reason": "Response B directly addresses the query by mentioning the Autumn Bazaar, Marie, and the farming aspect. It also includes the leaderboard competition and rewards, which are all mentioned in the query. Response A, while mentioning the Autumn Bazaar and Marie, does not directly address the farming aspect or the rewards and competition."
         },
         "accuracy": {
             "response": "AB",
             "reason": "Both responses are accurate as they provide information about the Autumn Bazaar, Marie, and the farming aspect. Response B adds more details about the rewards and competition, making it more comprehensive."
         },
         "clarity": {
             "response": "AB",
             "reason": "Both responses are clear as they use emojis and hashtags to make the caption more engaging and easier to understand. Response B adds more details and excitement to the caption, making it more 

In [61]:
print(text.split("###END")[0])


     {
         "relevance": {
             "response": "B",
             "reason": "Response B directly addresses the query by mentioning the Autumn Bazaar, Marie, and the farming aspect. It also includes the leaderboard competition and rewards, which are all mentioned in the query. Response A, while mentioning the Autumn Bazaar and Marie, does not directly address the farming aspect or the rewards and competition."
         },
         "accuracy": {
             "response": "AB",
             "reason": "Both responses are accurate as they provide information about the Autumn Bazaar, Marie, and the farming aspect. Response B adds more details about the rewards and competition, making it more comprehensive."
         },
         "clarity": {
             "response": "AB",
             "reason": "Both responses are clear as they use emojis and hashtags to make the caption more engaging and easier to understand. Response B adds more details and excitement to the caption, making it more 

In [38]:
import json

In [48]:
t2 = df['response'][1]
print(t2)

\n
     {  "relevance": {    "response": "A",    "reason": "Response A directly addresses the query by explaining that water is actually used in a bath tub for bathing purposes. It then provides reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. In contrast, Response B incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not relevant to the given query."  },  "accuracy": {    "response": "A",    "reason": "Response A accurately describes the purpose of a bath tub and the reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. Response B, however, contains inaccurate information as it incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not the case."  },  "clarity": {    "response": "A",    "reason": "Response A is clear and easy to understand as it directly addresses the query and provides rel

In [53]:
t3 = """
{  "relevance": {    "response": "A",    "reason": "Response A directly addresses the query by explaining that water is actually used in a bath tub for bathing purposes. It then provides reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. In contrast, Response B incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not relevant to the given query."  },  "accuracy": {    "response": "A",    "reason": "Response A accurately describes the purpose of a bath tub and the reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. Response B, however, contains inaccurate information as it incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not the case."  },  "clarity": {    "response": "A",    "reason": "Response A is clear and easy to understand as it directly addresses the query and provides relevant information. Response B, on the other hand, is unclear as it discusses the use of water as a conductor of electricity in a bath tub, which is not relevant to the given query."  },  "completeness": {    "response": "A",    "reason": "Response A covers all aspects of the query by explaining that water is used in a bath tub for bathing purposes and providing reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. Response B does not address the query completely as it focuses on an irrelevant topic."  },  "conciseness": {    "response": "A",    "reason": "Both responses are concise, but Response A is more to the point as it directly addresses the query. Response B, while concise, discusses an irrelevant topic."  },  "logical_flow": {    "response": "A",    "reason": "Response A presents information in a logical and organized manner by first addressing the query and then providing additional relevant information. Response B, however, lacks logical flow as it discusses an irrelevant topic."  },  "evidence_examples": {    "response": "A",    "reason": "Response A provides relevant evidence and examples to support its points, such as explaining that bath tubs are smaller than swimming pools or hot tubs and that they are not designed to be continuously filled with water. Response B does not provide relevant evidence or examples as it discusses an irrelevant topic."  },  "consistency": {    "response": "A",    "reason": "Response A maintains internal consistency without contradictions as it directly addresses the query and provides relevant information. Response B, however, is inconsistent as it discusses an irrelevant topic."  },  "objectivity": {    "response": "A",    "reason": "Both responses are objective, but Response A is more relevant to the given query."  },  "responsiveness": {    "response": "A",    "reason": "Response A is more responsive to the nuances and sub-questions within the query as it directly addresses the query and provides relevant information. Response B does not address the query or any sub-questions as it discusses an irrelevant topic."  }}
"""

In [54]:
t4 = json.loads(t3)
print(t4)

{'relevance': {'response': 'A', 'reason': 'Response A directly addresses the query by explaining that water is actually used in a bath tub for bathing purposes. It then provides reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. In contrast, Response B incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not relevant to the given query.'}, 'accuracy': {'response': 'A', 'reason': 'Response A accurately describes the purpose of a bath tub and the reasons why water in a bath tub is not used in the same way as in swimming pools or hot tubs. Response B, however, contains inaccurate information as it incorrectly assumes that the query is about using water as a conductor of electricity in a bath tub, which is not the case.'}, 'clarity': {'response': 'A', 'reason': 'Response A is clear and easy to understand as it directly addresses the query and provides relevant information. Response B, on the